In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import json
import rdflib, urllib
import pandas as pd
from owlready2 import *
from tqdm.notebook import tqdm
import os

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
with open("../qald10_id2label.json", "r") as f:
    id2label = json.load(f)

In [4]:
g = rdflib.Graph()
WD= rdflib.Namespace("http://www.wikidata.org/entity/")
g.bind("wd", WD)
WDT = rdflib.Namespace("http://www.wikidata.org/prop/direct/")
g.bind("wdt", WDT)

In [5]:
enti_list = []
prop_list = []
prop_str = ''
for id in list(id2label.keys()):
    if id.startswith('wdt:'):
        prop_list.append(id)
        prop_str += id + ', '
    else:
        enti_list.append(id)
prop_str = prop_str[:-2]
len(prop_list), len(enti_list)

(160, 483)

In [6]:
prop_str

'wdt:P61, wdt:P86, wdt:P69, wdt:P1303, wdt:P97, wdt:P2046, wdt:P2093, wdt:P35, wdt:P178, wdt:P473, wdt:P106, wdt:P206, wdt:P2925, wdt:P131, wdt:P1411, wdt:P2521, wdt:P115, wdt:P2048, wdt:P19, wdt:P179, wdt:P1542, wdt:P1622, wdt:P610, wdt:P119, wdt:P6509, wdt:P800, wdt:P108, wdt:P20, wdt:P425, wdt:P585, wdt:P172, wdt:P575, wdt:P735, wdt:P2936, wdt:P54, wdt:P2031, wdt:P674, wdt:P488, wdt:P3373, wdt:P26, wdt:P1034, wdt:P169, wdt:P1830, wdt:P176, wdt:P135, wdt:P2927, wdt:P22, wdt:P706, wdt:P1344, wdt:P40, wdt:P463, wdt:P452, wdt:P122, wdt:P740, wdt:P461, wdt:P156, wdt:P1365, wdt:P1412, wdt:P1120, wdt:P1342, wdt:P2522, wdt:P1098, wdt:P4969, wdt:P802, wdt:P1873, wdt:P1619, wdt:P1476, wdt:P527, wdt:P400, wdt:P1923, wdt:P1433, wdt:P4214, wdt:P17, wdt:P85, wdt:P186, wdt:P27, wdt:P47, wdt:P1441, wdt:P102, wdt:P2102, wdt:P937, wdt:P84, wdt:P157, wdt:P582, wdt:P840, wdt:P138, wdt:P1056, wdt:P144, wdt:P1448, wdt:P467, wdt:P451, wdt:P569, wdt:P162, wdt:P276, wdt:P495, wdt:P570, wdt:P3450, wdt:P25, w

In [7]:
if not os.path.exists('./tbl.ttl'):
  for item in tqdm(enti_list):
    item = item.split(':')[-1]
    query = \
    '''
    CONSTRUCT {
      wd:''' + item + ''' ?p ?o.
    }
    WHERE {
      wd:''' + item + ''' ?p ?o.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
      FILTER ( ?p IN (''' + prop_str + '''))
    }
    '''

    endpointUrl = 'https://query.wikidata.org/sparql'
    url = endpointUrl + '?query=' + urllib.parse.quote(query)

    g.parse(url)
else:
  g.parse('./tbl.ttl')

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x00000252868293A0>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\py39\lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\USER\anaconda3\envs\py39\lib\site-packages\isodate\isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
  File "c:\Users\USER\anaconda3\envs\py39\lib\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0220-01-01'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x00000252868293A0>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\

In [12]:
new_g = rdflib.Graph()
WD= rdflib.Namespace("http://www.wikidata.org/entity/")
new_g.bind("wd", WD)
WDT = rdflib.Namespace("http://www.wikidata.org/prop/direct/")
new_g.bind("wdt", WDT)

In [13]:
for item in tqdm(enti_list[:1]):
    item = item.split(':')[-1]
    query = \
    '''
    CONSTRUCT {
        wd:''' + item + ''' wdt:P31/wdt:P279* ?instanceOf.
    }
    WHERE {
        wd:''' + item + ''' wdt:P31/wdt:P279* ?instanceOf.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    '''

    endpointUrl = 'https://query.wikidata.org/sparql'
    url = endpointUrl + '?query=' + urllib.parse.quote(query)

    new_g.parse(url)

  0%|          | 0/1 [00:00<?, ?it/s]

HTTPError: HTTP Error 500: Internal Server Error

In [22]:
knows_query = """
SELECT DISTINCT ?x ?y ?z
WHERE {
    ?x ?y ?z.
}"""

triple_list = []
qres = g.query(knows_query)
for row in qres:
    triple_list.append([str(row[0]), str(row[1]), str(row[2])])
len(triple_list)

16853

In [8]:
knows_query = """
SELECT ?instance ?class
WHERE {
    ?instance wdt:P31 ?class.
}"""

qres = g.query(knows_query)

In [13]:
result = []
for row in qres:
    result.append(row[0])
    [row[1]]
len(set(result))

426

In [14]:
set(result), type:'T_i', id2labe[result[0]], tbox:[]

{rdflib.term.URIRef('http://www.wikidata.org/entity/P1082'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P1344'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P1352'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P1376'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P1410'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P1411'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P1552'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P166'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P17'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P175'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P194'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P2002'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P2043'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P2044'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P2046'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/P2048'),
 rdflib.term.URIRef('http://w

In [25]:
knows_query = """
SELECT ?x ?y ?z ?class ?class2
WHERE {
    ?x ?y ?z.
    ?x wdt:P31 ?class.
    ?z wdt:P31 ?class2.
}"""

qres = g.query(knows_query)

In [26]:
for row in qres:
    print(row[0], row[1], row[2], row[3], row[4])
    break

http://www.wikidata.org/entity/P2002 http://www.wikidata.org/prop/direct/P17 http://www.wikidata.org/entity/Q30 http://www.wikidata.org/entity/Q105388954 http://www.wikidata.org/entity/Q3624078


In [28]:
knows_query = """
SELECT ?instance1 ?p ?instance2 ?class1 ?class2
WHERE {
    ?instance1 ?p ?instance2.
    ?instance1 wdt:P31 ?class1.
    ?instance2 wdt:P31 ?class2.
}"""

qres = g.query(knows_query)

In [29]:
triple_list = []
for row in qres:
    p = str(row[1]).replace('http://www.wikidata.org/prop/direct/', 'wdt:')
    i1 = str(row[0]).replace('http://www.wikidata.org/entity/', 'wd:')
    i2 = str(row[2]).replace('http://www.wikidata.org/entity/', 'wd:')
    c1 = str(row[3]).replace('http://www.wikidata.org/entity/', 'wd:')
    c2 = str(row[4]).replace('http://www.wikidata.org/entity/', 'wd:')

    triple_list.append([i1, p, i2, c1, c2])
len(triple_list)

5669

In [30]:
triple_list

[['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q105388954', 'wd:Q3624078'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q105388954', 'wd:Q6256'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q105388954', 'wd:Q1520223'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q105388954', 'wd:Q512187'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q105388954', 'wd:Q5255892'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q105388954', 'wd:Q1489259'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q24239898', 'wd:Q3624078'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q24239898', 'wd:Q6256'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q24239898', 'wd:Q1520223'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q24239898', 'wd:Q512187'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q24239898', 'wd:Q5255892'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q24239898', 'wd:Q1489259'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q62589320', 'wd:Q3624078'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q62589320', 'wd:Q6256'],
 ['wd:P2002', 'wdt:P17', 'wd:Q30', 'wd:Q62589320', 'wd:Q1520223'],


In [9]:
df = pd.DataFrame(triple_list, columns=['S', 'P', 'O', 'domain', 'range'])

# the number of instance-level triples containing the property of the unit path
dpr_df = df.groupby(['domain', 'P', 'range'], as_index=False).S.count()

# the total number of triples from domain class to range class
dr_df = df.groupby(['domain', 'range'], as_index=False).S.count()

weight_df = pd.merge(df, dpr_df, on=['domain', 'P', 'range'], how='inner', suffixes=('','_dpr'))
weight_df = pd.merge(weight_df, dr_df, on=['domain', 'range'], how='inner', suffixes=('','_dr'))
weight_df['W'] = 1 - (weight_df['S_dpr'] / weight_df['S_dr'])

In [14]:
weight_df

,S,P,O,domain,range,S_dpr,S_dr,W
0,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q3624078,1,1,0.0
1,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q6256,1,1,0.0
2,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q1520223,1,1,0.0
3,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q512187,1,1,0.0
4,wd:P2002,wdt:P17,wd:Q30,wd:Q105388954,wd:Q5255892,1,1,0.0
...,...,...,...,...,...,...,...,...
5664,wd:Q30,wdt:P463,wd:Q1065,wd:Q1489259,wd:Q484652,1,1,0.0
5665,wd:Q30,wdt:P31,wd:Q6256,wd:Q1489259,wd:Q104098715,1,1,0.0
5666,wd:Q30,wdt:P31,wd:Q6256,wd:Q1489259,wd:Q15617994,1,1,0.0
5667,wd:Q30,wdt:P2936,wd:Q1860,wd:Q1489259,wd:Q34770,1,2,0.5


In [10]:
weight_df = weight_df[weight_df['domain'] != weight_df['range']]
final_df = weight_df[['domain', 'P', 'range', 'W']].drop_duplicates().reset_index(drop=True)
final_df

,domain,P,range,W
0,wd:Q105388954,wdt:P17,wd:Q3624078,0.0
1,wd:Q105388954,wdt:P17,wd:Q6256,0.0
2,wd:Q105388954,wdt:P17,wd:Q1520223,0.0
3,wd:Q105388954,wdt:P17,wd:Q512187,0.0
4,wd:Q105388954,wdt:P17,wd:Q5255892,0.0
...,...,...,...,...
3098,wd:Q1489259,wdt:P463,wd:Q484652,0.0
3099,wd:Q1489259,wdt:P31,wd:Q104098715,0.0
3100,wd:Q1489259,wdt:P31,wd:Q15617994,0.0
3101,wd:Q1489259,wdt:P2936,wd:Q34770,0.5


In [11]:
final_df.to_csv('../wiki_unit_path.csv', index=False)